# Projekt: zlomeniny způsobené pády pro věkovou kategorii 0-19 let

__[Odkaz na blogový článek](https://medium.com/@vackova.anna26/ztracené-roky-zdravého-života-v-důsledku-zlomenin-způsobených-pády-u-dětí-a-mladistvých-1a5fdc5aefc2)__

In [1]:
import pandas as pd

## Výběr nefatálních pádů pro věkovou kategorii 1-4 z originálního datasetu
- Vybíráme případy, které **nekončí umrtím a jejichž ošetření proběhlo ambulantně**, ať už s následnou péčí, či nikoliv
- Rozsah originálního datasetu je 34 693 990 řádků a 3,3 GB. Obsahem postihuje všechny úrazy evidované poskytovatelm zdravotní péče a vykázené k úhradě některé ze zdravotních pojišťoven za roky 2010 až 2022

### Průzkum dat

In [ ]:
data = pd.read_csv('urazy_originalni_dataset.csv', sep=',', nrows=1000, header=0, encoding="utf-8") # nacte jen 1000 radku ze souboru
display(data.head())
data.info()
data['umrti'].unique()

Na základě průzkumu a dřívější práce s daty i doprovodnými meateriály víme, že není mnoho sloupečků, které by měly null-hodnoty. Jedním z těch, který null hodnoty má, je sloupeček pod_kat obsahující příčnu úrazu. V datasetu je 15 196 253 vyplněných případů, u 19 497 737 případů o příčině úrazu nevíme.

Dataset nefatálních pádů ošetřených pouze ambulantně pro věkovou skupinu 0-19 let z originálního datasetu vyfilturjeme na základě 4 parametrů:
1. hodnota sloupečku pod_kat musí být W00-W19, což je kód pro úraz zapříčeněný pádem
2. hodnota sloupečku vek_kat musí obsahovat honoty od 1 do 4
3. hodnota sloupečku umrti musí být NULL
4. hodnota sloupečku vaznost nesmí být IIa, IIb, IIc, IId

### Získání menšího datasetu

In [5]:
falls_children_youth = pd.DataFrame()

for chunk in pd.read_csv('urazy_originalni_dataset.csv', chunksize=500000, encoding='utf-8', sep=','): 
    falls = chunk.query('pod_kat == "W00-W19" and vek_kat <= 4')
    falls_children_youth = pd.concat([falls_children_youth, falls], ignore_index=True)
falls_children_youth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2449240 entries, 0 to 2449239
Data columns (total 37 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   ID_urazu       int64 
 1   rok            int64 
 2   mesic          int64 
 3   pohlavi        int64 
 4   vek_kat        int64 
 5   kraj_pacient   object
 6   kraj_icz       object
 7   pod_kat        object
 8   polytrauma     int64 
 9   operace        int64 
 10  dlouha_jip     int64 
 11  nasledna_pece  int64 
 12  vaznost        object
 13  umrti          object
 14  S00_S09        int64 
 15  S10_S19        int64 
 16  S20_S29        int64 
 17  S30_S39        int64 
 18  S40_S49        int64 
 19  S50_S59        int64 
 20  S60_S69        int64 
 21  S70_S79        int64 
 22  S80_S89        int64 
 23  S90_S99        int64 
 24  T00_T07        int64 
 25  T08_T14        int64 
 26  T15_T19        int64 
 27  T20_T25        int64 
 28  T26_T28        int64 
 29  T29_T32        int64 
 30  T33_T35        int

In [6]:
falls_children_youth_outpatient = falls_children_youth.query('umrti.isnull() and vaznost not in ["IIa", "IIb", "IIc", "IId"]')
display(falls_children_youth_outpatient.info())
falls_children_youth_outpatient['vaznost'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Index: 2191259 entries, 0 to 2449239
Data columns (total 37 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   ID_urazu       int64 
 1   rok            int64 
 2   mesic          int64 
 3   pohlavi        int64 
 4   vek_kat        int64 
 5   kraj_pacient   object
 6   kraj_icz       object
 7   pod_kat        object
 8   polytrauma     int64 
 9   operace        int64 
 10  dlouha_jip     int64 
 11  nasledna_pece  int64 
 12  vaznost        object
 13  umrti          object
 14  S00_S09        int64 
 15  S10_S19        int64 
 16  S20_S29        int64 
 17  S30_S39        int64 
 18  S40_S49        int64 
 19  S50_S59        int64 
 20  S60_S69        int64 
 21  S70_S79        int64 
 22  S80_S89        int64 
 23  S90_S99        int64 
 24  T00_T07        int64 
 25  T08_T14        int64 
 26  T15_T19        int64 
 27  T20_T25        int64 
 28  T26_T28        int64 
 29  T29_T32        int64 
 30  T33_T35        int64 
 

None

vaznost
Ia    1155155
Ib    1036104
Name: count, dtype: int64

Z doprovodného přehledu víme, že originální dataset obsahuje 9 439 539 pádů, z toho 2 449 240 pro věkové kategorie 1-4, tedy 0-19 let. Žádný z těchto případů není fatální a 2 191 259 případů vyžadovalo ambulantní ošetření bez hospitalizace. Z toho 1 036 104 případů se nicméně neobešlo bez následné péče.

## Redukce nepotřebných sloupečků
Jelikož budeme na typ zranění navazovat další dvě proměnné hodnoty (disability weight a healing time), rozhodly jsme se, že zpracujeme pouze typ úrazu zlomenina, což je konec konců jedno z nejčastějších poranění, které děti a mladiství utrpí. Touto redukcí chceme dosáhnout větší přesnosti při budoucím výpočtu DALYs a pevně stanovit podmínky pro lepší komunikaci výsledku.


In [ ]:
display(falls_children_youth_outpatient.columns)
empty_columns_test = falls_children_youth_outpatient[['polytrauma', 'operace', 'dlouha_jip']].apply(pd.Series.value_counts) # zjistuju, jestli jsou tyhle sloupecky skutecne prazdne
print(empty_columns_test)

Index(['ID_urazu', 'rok', 'mesic', 'pohlavi', 'vek_kat', 'kraj_pacient',
       'kraj_icz', 'pod_kat', 'polytrauma', 'operace', 'dlouha_jip',
       'nasledna_pece', 'vaznost', 'umrti', 'S00_S09', 'S10_S19', 'S20_S29',
       'S30_S39', 'S40_S49', 'S50_S59', 'S60_S69', 'S70_S79', 'S80_S89',
       'S90_S99', 'T00_T07', 'T08_T14', 'T15_T19', 'T20_T25', 'T26_T28',
       'T29_T32', 'T33_T35', 'T36_T50', 'T51_T65', 'T66_T78', 'T79_T79',
       'T80_T88', 'T90_T98'],
      dtype='object')

   polytrauma  operace  dlouha_jip  umrti
0     2191259  2191259     2191259    NaN


Dle očekávání jsou sloupečky polytrauma, operace i dlouha_jip prazdné, neslučují se totiž s ošetřením bez nutnosti hospitalizace. Z dalších sloupečků zachovám jen ty relevantní a ty, které by mohly mít přínos pro budoucí vizualizaci. Z hlediska MKN-10 kódů pro diagnózy (S a T sloupečky) potřebuju zachovat jen veškeré S, které označují jednotlivé čáasti těla, a T00_T07 a T08_T14. Zbylé diagnózy nemohou být zlomeniny, jedná se například o otravy, popáleniny, omrzliny a v několika případech i o mnohočetná a neurčená poranění.

In [17]:
df_selected_columns = falls_children_youth_outpatient[
    [
        'ID_urazu', 'rok', 'mesic', 'pohlavi', 'vek_kat', 'kraj_pacient',
        'nasledna_pece', 'vaznost', 'S00_S09', 'S10_S19', 'S20_S29',
        'S30_S39', 'S40_S49', 'S50_S59', 'S60_S69', 'S70_S79', 'S80_S89',
        'S90_S99', 'T00_T07', 'T08_T14'
    ]
].copy()
df_selected_rows = df_selected_columns.query(
    'S00_S09 == 1 or S10_S19 == 1 or S20_S29 == 1 or S30_S39 == 1 or '
    'S40_S49 == 1 or S50_S59 == 1 or S60_S69 == 1 or S70_S79 == 1 or '
    'S80_S89 == 1 or S90_S99 == 1 or T00_T07 == 1 or T08_T14 == 1'
).copy()

df_selected_rows

,ID_urazu,rok,mesic,pohlavi,vek_kat,kraj_pacient,nasledna_pece,vaznost,S00_S09,S10_S19,S20_S29,S30_S39,S40_S49,S50_S59,S60_S69,S70_S79,S80_S89,S90_S99,T00_T07,T08_T14
0,33557641,2011,4,1,4,CZ099,1,Ib,0,0,0,0,0,0,0,0,0,1,0,0
1,33557659,2021,5,2,2,CZ053,1,Ib,0,0,0,0,0,0,0,0,0,1,0,0
2,33557688,2010,2,1,3,CZ080,0,Ia,1,0,0,0,0,0,0,0,0,0,0,0
3,33557698,2016,5,1,1,CZ080,0,Ia,0,0,0,0,0,0,0,0,1,0,0,0
4,33557701,2021,9,1,2,CZ080,0,Ia,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2449235,20969837,2011,2,2,4,CZ042,0,Ia,0,0,0,0,0,0,0,0,0,1,0,0
2449236,20969984,2018,8,1,2,CZ032,0,Ia,0,0,0,0,0,0,0,0,1,0,0,0
2449237,20969987,2022,7,1,3,CZ032,1,Ib,1,0,0,0,0,0,0,0,0,0,0,0
2449238,20970160,2016,9,1,2,CZ020,0,Ia,0,0,0,0,0,0,0,0,0,1,0,0


Výsledkem je tabulka o 19 sloupečcách a 2 151 406 řádcích.

## Diagnózy
Jako další krok bude potřeba zmenšit tabulky diagnóz, aby korespondovaly s naší výsečí dat a dostat se ideálně na co nejpřesnější hodnotu MKN, ke které dohledáme konstanty disability weights.

### Diagnózy S
- Původní velikost 34 716 517 řádků a 2,41 GB
- Váže se na dataset úrazů a poskytuje detailnější infomrace o poranění

In [10]:
input = pd.read_csv('diagnozy_s.csv', nrows=1000, encoding='utf-8', sep=',')
input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   ID_urazu                                           1000 non-null   int64 
 1   MKN2                                               1000 non-null   object
 2   povrchni_poraneni                                  1000 non-null   int64 
 3   otevrena_rana                                      1000 non-null   int64 
 4   zlomenina                                          1000 non-null   int64 
 5   vymknuti_podvrtnuti_a_nataženi_kloubu_a_vazu       1000 non-null   int64 
 6   poraneni_nervu                                     1000 non-null   int64 
 7   poraneni_krevnich_cev                              1000 non-null   int64 
 8   poraneni_svalu_a_slachy                            1000 non-null   int64 
 9   nitrolebni_poraneni 

Soubor zpracujeme ve 3 krocích:
1. Zajístíme, aby výsledná tabulka obsahovala jen ID_urazu korespondující s naším pracovním datasetem
2. Vybereme si jen ty řádky, které mají hodnotu 1 (True) pro typ poranění zlomenina
3. Zbavíme se nepotřebných sloupčeků

In [18]:
diagnosis_s = pd.DataFrame()
for chunk_s in pd.read_csv('diagnozy_s.csv', chunksize=500000, encoding='utf-8', sep=','):
    filter_s = chunk_s['ID_urazu'].isin(df_selected_rows['ID_urazu']) & (chunk_s['zlomenina'] == 1)
    filtered_chunk_s = chunk_s[filter_s]
    diagnosis_s = pd.concat([diagnosis_s, filtered_chunk_s[['ID_urazu', 'MKN2', 'zlomenina']]], ignore_index=True)
display(diagnosis_s['MKN2'].value_counts())
diagnosis_s.info()

MKN2
S5 Poranění lokte a předloktí                                  154877
S6 Poranění zápěstí a ruky                                     115024
S9 Poranění kotníku a nohy pod ním                              67729
S4 Poranění ramene a paže (nadloktí)                            66005
S8 Poranění kolena a bérce                                      55600
S0 Poranění hlavy                                               20058
S7 Poranění kyčle a stehna                                       6691
S2 Poranění hrudníku                                             5536
S3 Poranění břicha‚ dolní části zad‚ bederní páteře a pánve      4416
S1 Poranění krku                                                  478
Name: count, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496414 entries, 0 to 496413
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   ID_urazu   496414 non-null  int64 
 1   MKN2       496414 non-null  object
 2   zlomenina  496414 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 11.4+ MB


Výsledkem je 496 414 záznamů. V dalším kroku odebereme sloupeček zlomenina, protože všechny řádky v našem novém datasetu S diagnóz jsou zlomeniny. Zároveň nahradíme hodnoty ze sloupečku MKN2 za konkrétní kódy. S0 až S9 označuje postiženou část těla a zlomenina má v číselníku MKN-10 vždy číslo 2, potřebujeme tedy zachovat 2 první znaky v buňkách sloupečku MKN2 a přidat k nim číslici 2.

In [12]:
diagnosis_s_to_mkn_code = diagnosis_s.drop(columns=['zlomenina'])
diagnosis_s_to_mkn_code['MKN2'] = diagnosis_s_to_mkn_code['MKN2'].str[:2] + '2'

In [9]:
diagnosis_s_to_mkn_code.to_csv('diagnozy_s_zlomeniny.csv', sep=',', index=False, encoding='utf-8')

### Diagnózy T
- Původní velikost 7 223 230 řádků a 364 MB
- Z hlediska zlomenin obsahuje záznamy o mnohočetných a blíže neurčených zlomeninách

Víme, že Diagnózy T taky obsahují zlomeniny, nicméně může se ukázat, že jsou popisy poranění příliš vágní, nebo je poranění příliš komplexní pro připojení k našemu výzkumu.

In [19]:
diagnosis_t = pd.DataFrame()
for chunk_t in pd.read_csv('diagnozy_t.csv', chunksize=500000, encoding='utf-8', sep=','):
    id_filter = chunk_t['ID_urazu'].isin(df_selected_rows['ID_urazu'])
    mkn10_filter = chunk_t['MKN10'].str[:3].isin(["T02", "T08", "T10", "T12"])
    filter_t = id_filter & mkn10_filter
    diagnosis_t = pd.concat([diagnosis_t, chunk_t[filter_t]], ignore_index=True)
display(diagnosis_t.info())    
diagnosis_t['MKN10'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 708 entries, 0 to 707
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID_urazu  708 non-null    int64 
 1   MKN10     708 non-null    object
dtypes: int64(1), object(1)
memory usage: 11.2+ KB


None

array(['T02 Zlomeniny postihující více částí těla',
       'T0200 Zlomeniny hlavy s krkem; zavřená',
       'T0210 Zlomeniny postihující hrudník s dolní částí zad a pánve; zavřená',
       'T022 Zlomeniny postihující více částí jedné horní končetiny',
       'T0220 Zlomeniny více částí jedné horní končetiny; zavřená',
       'T0221 Zlomeniny více částí jedné horní končetiny; otevřená',
       'T023 Zlomeniny postihující více částí jedné dolní končetiny',
       'T0230 Zlomeniny více částí jedné dolní končetiny; zavřená',
       'T024 Zlomeniny postihující více částí obou horních končetin',
       'T0240 Zlomeniny více částí obou horních končetin; zavřená',
       'T0250 Zlomeniny více částí obou dolních končetin; zavřená',
       'T0260 Zlomeniny více částí HK s DK; zavřená',
       'T0261 Zlomeniny více částí HK s DK; otevřená',
       'T028 Zlomeniny postihující jinak kombinované části těla',
       'T0280 Zlomeniny jinak kombinovaných části těla; zavřená',
       'T029 Mnohočetné zl

 Vzniklých 708 hodnot dále očistíme o mnohočetné a otevřené zlomeniny. Důvodem je, že otevřené zlomeniny a mnohočetná poranění představují takový druh komorbidity, pro který nebudeme schopné na základě dostupných infomrací dopočítat průměrnou dobu uzdravení a disability weight.

In [27]:
diagnosis_t_filtered = diagnosis_t[diagnosis_t['MKN10'].str.contains('otevřená|oteřená|zlomeniny|Zlomeniny')==False]
display(diagnosis_t_filtered.info())
diagnosis_t_filtered['MKN10'].unique()

<class 'pandas.core.frame.DataFrame'>
Index: 502 entries, 183 to 702
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID_urazu  502 non-null    int64 
 1   MKN10     502 non-null    object
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


None

array(['T08 Zlomenina páteře, úroveň neurčena',
       'T080 Zlomenina páteře, úroveň neurčena; zavřená',
       'T10 Zlomenina horní končetiny, úroveň neurčena',
       'T100 Zlomenina horní končetiny, úroveň neurčena; zavřená',
       'T12 Zlomenina dolní končetiny, úroveň neurčena',
       'T120 Zlomenina dolní končetiny, úroveň neurčena; zavřená'],
      dtype=object)

In [28]:
diagnosis_t_filtered.to_csv('diagnozy_t_zlomeniny.csv', sep=',', index=False, encoding='utf-8')

## Další redukce hlavního datasetu na základě profiltrovaných diagnóz

Po zpracování datasetů diagnóz se vracíme zpět k hlavnímu datasetu úrazů. Tím, že jsme si v datasetech diagnóz vybraly pouze případy splňující definované parametry, můžeme na základě ID_urazu zpětně zmenšit hlavní dataset.

In [46]:
filter = df_selected_rows['ID_urazu'].isin(diagnosis_s['ID_urazu']) | df_selected_rows['ID_urazu'].isin(diagnosis_t_filtered['ID_urazu'])
injuries_children_youth_fractures = df_selected_rows[filter]
injuries_children_youth_fractures.shape

(472157, 20)

In [47]:
injuries_children_youth_fractures.to_csv('pady_deti_mladistvi_fraktury.csv', sep=',', index=False, encoding='utf-8')

### Update: Dodatečné vyloučení T diagnóz a přepis hlavního datasetu

Po domluvě s projektovou parťačkou jsme se rozhodly diagnózy T z našeho výzkumu zcela vyloučit. Popis poranění není dost specifický.


In [48]:
df_including_t_diagnosis = pd.read_csv('pady_deti_mladistvi_fraktury.csv', sep=',', encoding='utf-8')
s_diagnosis = pd.read_csv('diagnozy_s_zlomeniny.csv', sep=',', encoding='utf-8')
filter = df_including_t_diagnosis['ID_urazu'].isin(s_diagnosis['ID_urazu'])
injuries_children_youth_fractures_s = df_including_t_diagnosis[filter].drop(columns=['T00_T07', 'T08_T14'])
injuries_children_youth_fractures_s.shape

(472000, 18)

In [ ]:
injuries_children_youth_fractures_s.to_csv('pady_deti_mladistvi_fraktury.csv', sep=',', index=False, encoding='utf-8')